<a href="https://colab.research.google.com/github/Love1117/Machine_learning-Projects/blob/main/Api%20Notebook%20Structure/API_Colab_Running_Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastapi uvicorn pyngrok nest-asyncio

In [2]:
import joblib
from google.colab import drive
drive.mount("/content/drive")

path = "/content/drive/My Drive/Models/insurance.joblib"

model = joblib.load(path)

Mounted at /content/drive


In [3]:
from fastapi import FastAPI
import numpy as np
from pydantic import BaseModel
from enum import Enum

app = FastAPI()

class Base(BaseModel):
  age: float
  sex: float
  bmi: float
  children: float
  smoker: float
  region: float
  charges: float


@app.get("/")
async def get_message():
  return {"message": "Welcome to Loveday Machine Learning Api"}


food_items = {"Nigeria":["Eba","Ewedu","Rice and beans","garri and soup"],
              "Rwanda": ["Isabela","Gigawa","Friday","Chike"],
              "Uganda": ["Peas","yolo","Temper","Loveday"],
               "Ghana": ["Petawa","Zimba","Police","Huncho"]}

class Available_cuisine(str, Enum):
  Nigeria = "Nigeria"
  Rwand = "Rwanda"
  Uganda = "Uganda"
  Ghana = "Ghana"

@app.get("/items/{cuisine}")
async def get_items(cuisine: Available_cuisine):
  return f"for {cuisine.value}, the available items are: {food_items[cuisine.value]}"


discounts = {1: "20% Discount",
             2: "40% Discount",
             3: "60% Discount",
             4: "80% Discount",
             5: "100% Discount"}

@app.get("/the_items")
async def the_items(discount: int):
  if discount > 5:
    return "please choose a range from number 1 to 5"
  else:
    return f"for your chosen number {discount}, your discount is: {discounts[discount]}"


items = []
@app.post("/item")
async def your_item(list_item: str):
  items.append(list_item)
  return items


@app.get("/put_num/{number}")
async def put_num(number: int):
  if number < len(items):
    return f"The word in the list for your selected index {number} is: {items[number]}"
  else:
    return f"your selected number {number} is out of range, please choose between 0 to {len(items)}"


@app.get("/list_of_words/{limit}")
async def list_of_words(limit: int = len(items)):
  return f"In the items these are the list of words for your limit: {items[0:limit]}"


@app.post("/predict")
async def predict(data: Base):
  data = data.dict()
  age = data["age"]
  sex = data["sex"]
  bmi = data["bmi"]
  children = data["children"]
  smoker = data["smoker"]
  region = data["region"]
  charges = data["charges"]

  classifier = model.predict([[age,sex,bmi,children,smoker, region, charges]])

  if (classifier[0] >= 0.5):
    return "Prediction: Has Insurance"
  else:
    return "Prediction: Has no Insurance"

In [4]:
import nest_asyncio
nest_asyncio.apply()

import uvicorn
import asyncio

# Create a Uvicorn configuration
config = uvicorn.Config(app, host="0.0.0.0", port=8000, loop="asyncio")

# Create a Uvicorn server instance
server = uvicorn.Server(config)

# Run the server in a separate asyncio task to avoid blocking the notebook
# This allows other cells (like ngrok) to execute.
asyncio.create_task(server.serve())

<Task pending name='Task-1' coro=<Server.serve() running at /usr/local/lib/python3.12/dist-packages/uvicorn/server.py:69>>

In [5]:
from pyngrok import ngrok

ngrok.set_auth_token("375mWYaMXwrlkyG1PLQUSKPGCTn_7TEuMBd5te7GFBfgjCQnZ")

public_url = ngrok.connect(8000)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://untelevised-spangly-lottie.ngrok-free.dev" -> "http://localhost:8000"
